---
title       : "Coding for Performance"
subtitle    : 
author      : Paul Schrimpf
date        : `j using Dates; print(Dates.today())`
bibliography: "perf.bib"
---

[![](https://i.creativecommons.org/l/by-sa/4.0/88x31.png)](http://creativecommons.org/licenses/by-sa/4.0/)

This work is licensed under a [Creative Commons Attribution-ShareAlike
4.0 International
License](http://creativecommons.org/licenses/by-sa/4.0/) 


### About this document {-}

This document was created using Weave.jl. The code is available in
[on github](https://github.com/schrimpf/ARGridBootstrap.jl). The same
document generates both static webpages and associated (jupyter
notebook)[argridboot.ipynb]. 

# Introduction

Today we will look into some methods to improve the speed of our
code. Although speed is sometimes important, never forget that speed
should be low on your list of priorities when writing code. You should
prioritize correctness and maintainability ahead of
performance. Nonetheless, performance does matter for some problems. 

If you have not already, be sure to read [the Peformance Tips section of Julia Docs](https://docs.julialang.org/en/v1/manual/performance-tips/#man-performance-tips-1). 

Also, read Rackauckas's notes on ["Optimizing Serial Code."](https://mitmath.github.io/18337/lecture2/optimizing)[@rackauckas2019a]. 

# Grid bootstrap

As a motivating example we will look at the gridded bootstrap of
Hansen (1999)[@hansen99]. 

<!-- FIXME: DESCRIPTION.  -->

Gauss, Matlab, and R code implementing Hansen's method is available on
[Hansen's
website](https://www.ssc.wisc.edu/~bhansen/progs/restat_99.html). The
Julia code below is more or less a direct translation from Hansen's R
code. Since this is a translation from R of a translation from Gauss,
this code will not necessarily follow best practices for Julia.

In [1]:
using ARGridBootstrap

function functiontext(functionname, filename; includedoc=true)
  lines = readlines(filename)
  fstart=findfirst(occursin.(Regex("function\\s+$(functionname)"),lines))
  fend  =fstart + findfirst(occursin.(r"^end",lines[(fstart+1):end]))  
  if (includedoc && occursin(r"^\"\"\"",lines[fstart-1]) )
    dend = fstart -1
    dstart = dend - findfirst(occursin.(r"^\"\"\"", lines[(fstart-2):(-1):1]))
  end
  lines[dstart:fend]
end

┌ Info: Recompiling stale cache file /home/paul/.julia/compiled/v1.2/ARGridBootstrap/7AB93.ji for ARGridBootstrap [8c7cac60-37ca-418c-be28-fd9d1bae9cd2]
└ @ Base loading.jl:1240


functiontext (generic function with 1 method)

In [2]:
println.(functiontext("b_est_original",joinpath(dirname(Base.pathof(ARGridBootstrap)), "ar.jl")));
println.(functiontext("ar1_original",joinpath(dirname(Base.pathof(ARGridBootstrap)),"ar.jl")));

"""
    b_est_original(y)

Estimate AR(1) model with intercept and time trend

    y[t] = θ[0] + θ[1]t + θ[2]y[t-1] + e[t]

# Arguments
- `y`: vector 

# Returns 
- `θ`: estimated coefficients
- `se`: standard errors
- `e`: residuals 
"""
function b_est_original(yin)
  T = length(yin)
  x = [ones(T-1) 2:T yin[1:(T-1)]]
  y = yin[2:T]
  θ = x'*x \ x'y
  e = y - x*θ
  se = sqrt.(diag(inv(x'*x) *(e'*e))./(T-4))
  (θ=θ,se=se,e=e)
end
"""
    ar1_original(y0, a, e, rindex=T->rand(1:length(e), T))

Simulate AR1 model by sampling errors from e with replacement. 
  
    y[t] = a*y[t-1] + ϵ[t]

# Arguments 
- `y0`: initial value for `y`
- `a`: AR parameter
- `e`: values of for error term. `ϵ = e[rindex(T)]]`
- `rindex` function that returns random index in 1:length(e)
  
# Returns 
- `y`: vector of length `T = length(e)`
"""
function ar1_original(y0, a, e, rindex=T->rand(1:length(e),T))
  T = length(e)
  y = Array{eltype(e)}(undef, T)
  y[1] = abs(a)<1 ? y0 : zero(eltype(y))
  et = e[rindex(T-1

In [3]:
println.(functiontext("gridbootstrap",joinpath(dirname(Base.pathof(ARGridBootstrap)), "gridbootstrap.jl")));

"""
    gridbootstrap(estimator, simulator, 
                  grid::AbstractVector, 
                  nboot=199)
  
Computes grid bootstrap estimates a single parameter model. 

For each α ∈ grid, repeatedly simulate data with parameter α and then compute an estimate. 
 

# Arguments
- `estimator` function of output of `simulator` that returns a
        2-tuple containing an estimate of α and its standard error.  
- `simulator` function that given `α` simulates data that can be used to estimate α
- `grid` grid of parameter values. For each value, `nboot`
        datasets will be simulated and estimates computed.  
- `nboot` 

# Returns
- `ba` hatα - α for each grid value and simulated dataset
- `t` t-stat  for each grid value and simulated dataset
"""
function gridbootstrap(estimator, simulator,
                       grid::AbstractVector,
                       nboot=199)
  g = length(grid)
  bootq = zeros(nboot, g)
  ba    = zeros(nboot, g)
  bootse = zeros(nboot,g)
  for ak in 1:g

## Improving performance

Now, let's run this code and time it. Note that we are running this
with only 50 grid points and 199 bootstrap replications. In real use,
you would want more like 999 bootstrap replications or more, and perhaps more
grid points.

In [4]:
# simulate some data
using Random, BenchmarkTools, Profile
T = 200
e = randn(T)
y0 = 0
a = 0.9
y = ar1_original(y0, a, e)
est = b_est_original(y)
αgrid = 0.84:(0.22/50):1.06
nboot= 199
wrapper(b_est) = function(x)
  out=b_est(x)
  (out.θ[3], out.se[3])
end
@btime (b,t) = gridbootstrap(wrapper(b_est_original), a->ar1_original(y0, a, est.e),
                             αgrid, nboot);

  545.383 ms (365375 allocations: 222.07 MiB)


To make code faster, we should begin by profiling.

In [5]:
Profile.clear();
Profile.init(n=10^7,delay=0.0001);
@profile (b,t) = gridbootstrap(wrapper(b_est_original), a->ar1_original(y0, a, est.e),
                               αgrid, 999);
Profile.print(noisefloor=2.0)

8918 ./task.jl:268; (::getfield(IJulia, Symbol("##15#18")))()
 8918 /home/paul/.julia/packages/IJulia/fRegO/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
  8918 ./essentials.jl:789; invokelatest
   8918 ./essentials.jl:790; #invokelatest#1
    8918 /home/paul/.julia/packages/IJulia/fRegO/src/execute_request.jl:67; execute_request(::ZMQ.Socket, ::IJulia.Msg)
     8918 /home/paul/.julia/packages/SoftGlobalScope/cSbw5/src/SoftGlobalScope.jl:218; softscope_include_string(::Module, ::String, ::String)
      8918 ./boot.jl:330; eval
       8879 /home/paul/.julia/dev/ARGridBootstrap/src/gridbootstrap.jl:32; gridbootstrap(::getfield(Main, Symbol("##4#5")){typeof(b_est_original)}, ::getfield(Main, Symbol("##9#10")), ::StepRangeLen{...
        8666 ./In[4]:12; (::getfield(Main, Symbol("##4#5")){typeof(b_est_original)})(::Array{Float64,1})
         8517 /home/paul/.julia/dev/ARGridBootstrap/src/ar.jl:50; b_est_original(::Array{Float64,1})
          8493 /build/julia/src/julia-1.2.0/usr/share/julia/

Profile.jl works very simply. Every 0.0001 seconds, the line of code
being executed gets recorded. `Profile.print` shows the count of how
many times each line of code got recorded. From the output (these
numbers can vary quite a bit from run to run), we see
there were 640 ticks in ` gridbootstrap_original` (exact numbers will
vary on each execution, but relative ones should be similar), and
almost all of these occurred within `inv`.  If we want the
code to be faster, we should focus on these lines.  Calling both `inv`
and `\` is redundant; we should combine these computations.

In [6]:
println.(functiontext("b_est_mldivide",joinpath(dirname(Base.pathof(ARGridBootstrap)), "ar.jl")));

"""
    b_est_mldivide(y)

  Estimate AR(1) model with intercept and time trend. 

    y[t] = θ[0] + θ[1]t + θ[2]y[t-1] + e[t]

# Arguments    
- `y`: vector 

# Returns
- `θ`: estimated coefficients
- `se`: standard errors
- `e`: residuals 
"""
function b_est_mldivide(yin)
  T = length(yin)
  x = [ones(T-1) 2:T yin[1:(T-1)]]
  y = yin[2:T]
  tmp = x'*x \ [x'*y I]
  θ = tmp[:,1]
  ixx = tmp[:,2:4]
  e = y - x*θ
  se = sqrt.(diag(ixx *(e'*e))./(T-4))
  (θ=θ,se=se,e=e)
end


In [7]:
@btime (b,t) = gridbootstrap(wrapper(b_est_mldivide), a->ar1_original(y0, a, est.e),
                             αgrid, nboot);

  116.672 ms (385673 allocations: 208.60 MiB)


From this, we get a speedup by about a factor of 4 on my computer.

In [8]:
Profile.clear();
@profile (b,t) = gridbootstrap(wrapper(b_est_mldivide), a->ar1_original(y0, a, est.e),
                               αgrid, 999);
Profile.print(noisefloor=2.0)

445 ./task.jl:268; (::getfield(IJulia, Symbol("##15#18")))()
 445 /home/paul/.julia/packages/IJulia/fRegO/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
  445 ./essentials.jl:789; invokelatest
   445 ./essentials.jl:790; #invokelatest#1
    445 /home/paul/.julia/packages/IJulia/fRegO/src/execute_request.jl:67; execute_request(::ZMQ.Socket, ::IJulia.Msg)
     445 /home/paul/.julia/packages/SoftGlobalScope/cSbw5/src/SoftGlobalScope.jl:218; softscope_include_string(::Module, ::String, ::String)
      445 ./boot.jl:330; eval
       405 /home/paul/.julia/dev/ARGridBootstrap/src/gridbootstrap.jl:32; gridbootstrap(::getfield(Main, Symbol("##4#5")){typeof(b_est_mldivide)}, ::getfield(Main, Symbol("##14#15")), ::StepRangeLen...
        261 ./In[4]:12; (::getfield(Main, Symbol("##4#5")){typeof(b_est_mldivide)})(::Array{Float64,1})
         113 /home/paul/.julia/dev/ARGridBootstrap/src/ar.jl:72; b_est_mldivide(::Array{Float64,1})
          97 ./abstractarray.jl:1313; hcat
           95 ./abstractarr

Now, the most time consuming parts of the code are, unsurprisingly,
the call to ` \`, and, perhaps surprisingly, ` hcat` from
creating ` x`. Allocating and copying memory is relatively slow. The
creation of ` x` involves both. We can avoid creating `x` by just
accumulating $X'y$ and $X'X$ in a loop.

In [9]:
println.(functiontext("b_est_nox",joinpath(dirname(Base.pathof(ARGridBootstrap)), "ar.jl")));

""" 
    b_est_nox(y)

  Estimate AR(1) model with intercept and time trend. 

    y[t] = θ[0] + θ[1]t + θ[2]y[t-1] + e[t]

# Arguments
- `y`: vector 

# Returns
- `θ`: estimated coefficients
- `se`: standard errors
- `e`: residualas 
"""
function b_est_nox(yin)
  T = length(yin)
  xx = zeros(eltype(yin),3,3)
  xy = zeros(eltype(yin),3)
  @inbounds @simd for t in 2:T
    xx[1,3] += yin[t-1]
    xx[2,3] += t*yin[t-1]
    xx[3,3] += yin[t-1]^2
    xy[1] += yin[t]
    xy[2] += t*yin[t]
    xy[3] += yin[t-1]*yin[t]
  end 
  xx[1,1] = T-1 # = 1'*1
  xx[1,2] = xx[2,1] = (T+1)*T/2 - 1 # sum(p+1:T)
  xx[2,2] = (2*(T)+1)*(T)*(T+1)/6 - 1 # sum((p+1:T).^2)  
  xx[3,1] = xx[1,3]
  xx[3,2] = xx[2,3]
  tmp = xx \ [xy I]
  θ = tmp[:,1]
  ixx = tmp[:,2:4]
  e = similar(yin,T-1)
  @simd for t in 2:T
    @inbounds e[t-1] = yin[t] - θ[1] - θ[2]*t - θ[3]*yin[t-1]
  end
  se = sqrt.(diag(ixx *(e'*e))./(T-4))
  (θ=θ,se=se,e=e)
end


In [10]:
@btime (b,t) = gridbootstrap(wrapper(b_est_nox), a->ar1_original(y0, a, est.e),
                             αgrid, nboot);

  66.517 ms (304481 allocations: 89.82 MiB)


We have further cut the time by a factor of two. However, this performance
optimization has been costly in terms of readability and extensibility
of our code. If we wanted to fit an AR(p) model instead of AR(1), the
` b_est_nox` function would be more difficult to modify than the
` b_est_mldivide` version. 

EXERCISE: Read [the Performance Tips section of Julia
Manual](https://docs.julialang.org/en/v1/manual/performance-tips/) and
incorporate some of these tips into the above code.

EXERCISE: write a version of ` b_est` that avoids allocating the full
$T \times 3$ $X$ matrix, but can still be generalized to an AR(p) model. 

EXERCISE: examine how the relative performance of these versions of `
b_est` vary with ` T`, ` nboot`, and the number of grid points. 

EXERCISE: the Julia package ` StaticArrays.jl` provides an alternative
array implementation that is often much faster than ` Base.Array`. Try
implementing ` b_est` using ` StaticArrays.jl`. You will likely need to
use mutable arrays (see ` @MMatrix` and ` @MVector`). Note that ` inv` of
a small array will be substantially faster when using ` StaticArray.jl`
instead of ` Base.Array`. 

## Fastest version

The fastest version of the code that I could write combines the ideas
above. As above, it avoids allocating `x`. It also avoids allocating
`e` by combining the simulation and estimation into a single
loop. Finally, it uses mutable static arrays to ensure that operations
on `xx` and `xy` have as little overhead as possible. Note that for
small StaticArrays, `inv` will call a specialized, fast version, and
ends up being faster than `\`.

In [11]:
using StaticArrays

In [12]:
println.(functiontext("simulate_estimate_arp",joinpath(dirname(Base.pathof(ARGridBootstrap)), "ar.jl")));

"""
    simulate_estimate_arp(y0, a, e, ar::Val{P}, rindex=T->rand(1:length(e),T)) 

Simulates and estimates an AR(P) model. `y` is simulated as
  
   y[t] = a*y[t-1] + ϵ[t]
  
and the estimate of θ from 

   y[t] = θ[1] + θ[2]t + θ[3] y[t-1] + ... + θ[P] y[t-P] + u[t] 

is computed. 

# Arguments
- `y0` initial value of y
- `a` AR(1) parameter
- `e` error terms to sample from `ϵ[t] = e[rindex(1)]`
- `ar::Val{P}` order of autoregressive model to estimate
- `rindex` function that returns random index in 1:length(e)

# Returns
- `θ` estimated coefficients
- `se` standard errors
"""
function simulate_estimate_arp(y0, a, e, ar::Val{P}=Val(1),
                               rindex=()->rand(1:length(e))) where P
  T = length(e)
  length(a)==P || error("length(a) not equal to P")
  xx = @MMatrix zeros(eltype(e),P+2, P+2)
  xy = @MVector zeros(eltype(e),P+2)
  yy = zero(eltype(e))
  xt = @MVector ones(eltype(e), P+2)
  if (abs(a)<1)
    xt[3:(P+2)] .= y0
  else 
    xt[3:(P+2)] .= 0.0
  end
  

In [13]:
estimator(y0=y0,e=est.e) = function(a) 
  out = simulate_estimate_arp(y0,a,e)
  (out.θ[3], out.se[3])
end
@btime  (b,t) = gridbootstrap(estimator(), a->a, αgrid, nboot);

  26.167 ms (111651 allocations: 4.80 MiB)


On my computer, this version of the code is about 15 times faster than
the original. 


# Multi-threading

Modern computers almost all have multiple cores. We can divide the
time it takes our code by up to the number of cores we have (but
usually less) by writing multi-threaded code. Multi-threaded code
performs multiple tasks at once with shared memory. Before you begin
writing multi-threaded code, you should make sure your code isn't
already using all available cores. It is likely that the BLAS and
Lapack libraries that Julia uses for linear algebra are
multi-threaded. If you code is dominated by large matrix operations,
it may already be using all available cores. In that case, there will
not be much benefit from additional multi-threading.

Read ["The Basics of Single Node Parallel Computing"](https://mitmath.github.io/18337/lecture5/parallelism_overview)
Rackauckus (2019)[@rackauckus2019b].

Once we have decided that the code might benefit from multi-threading,
we should look for loops (or other independent tasks) that can be
multi-threaded. There is some overhead from creating threads and
communicating among them. Multi-threading generally works best for
loops where each iteration involves substantial work, and each
iteration is independent of all others. The loops over grid points and
bootstrap repetitions in ` gridbootstrap` are perfect candidates. We
don't care about the order in which these loops get executed. The
result of each iteration is (mostly) independent of all others.

Some care must be taken with random number generators and
multi-threaded code. See
[the Julia docs](https://docs.julialang.org/en/v1/manual/parallel-computing/index.html#Side-effects-and-mutable-function-arguments-1) for more information.

In [14]:
println.(functiontext("gridbootstrap_threaded",joinpath(dirname(Base.pathof(ARGridBootstrap)), "gridbootstrap.jl")));

"""
    gridbootstrap_threaded(estimator, simulator, 
                    grid::AbstractVector, 
                    nboot=199)
    
Computes grid bootstrap estimates a single parameter model. 

For each α ∈ grid, repeatedly simulate data with parameter α and then compute an estimate. 
 

# Arguments
- `estimator` function of output of `simulator` that returns a
    2-tuple containing an estimate of α and its standard error.  
- `simulator` function that given `α` and `rng`, simulates data
    that can be used to estimate α 
- `grid` grid of parameter values. For each value, `nboot`
    datasets will be simulated and estimates computed.  
- `nboot` 

# Returns
- `ba` hatα - α for each grid value and simulated dataset
- `t` t-stat  for each grid value and simulated dataset
"""
function gridbootstrap_threaded(estimator, simulator,
                                grid::AbstractVector,
                                nboot=199; rng=rngarray(nthreads()))
  g = length(grid)
  bootq = zeros(n

In [15]:
println.(functiontext("rngarray",joinpath(dirname(Base.pathof(ARGridBootstrap)), "gridbootstrap.jl")));

"""
    rngarray(n)

  Create `n` rng states that will not overlap for 10^20 steps.

  Note: this will be unneeded in Julia 1.3 when thread-safe RNG is
  included.
"""
function rngarray(n)
  baserng =  MersenneTwister()
  rng = Array{typeof(baserng)}(undef, Base.Threads.nthreads())
  rng[1] = baserng
  steps = big(10)^20 # randjump is precomputed for steps = big(10)^20
  for i in 2:nthreads()
    rng[i] = Future.randjump(rng[i-1], steps) 
  end
  rng
end


Now, let's try multi-threading the original version of the code.

In [16]:
using Base.Threads
println("Single thread, original version")
@time begin # this is so slow that using btime is not so necessary
  (b,t) = gridbootstrap(wrapper(b_est_original), a->ar1_original(y0, a, est.e),
                        αgrid, 199);
end;

rng = rngarray(nthreads())
# make sure the threaded version is compiled before timing it
(b,t) = gridbootstrap_threaded(wrapper(b_est_original),
                               (a, rng)->ar1_original(y0, a, est.e, n->rand(rng,1:(T-1),n)),
                               αgrid, 2, rng=rng);
println("$(nthreads()) threads, original version")
@time begin # this is so slow that using btime is not so necessary
  (b,t) = gridbootstrap_threaded(wrapper(b_est_original),
                                 (a, rng)->ar1_original(y0, a, est.e, n->rand(rng,1:(T-1),n)),
                                 αgrid, 199, rng=rng);
end;

Single thread, original version
  0.652117 seconds (428.66 k allocations: 225.018 MiB, 4.02% gc time)
30 threads, original version
 17.485715 seconds (6.35 M allocations: 313.294 MiB, 0.71% gc time)


The execution times are nearly identical on my computer. The reason is
that the computation is dominated by the creation of ` X` and
multiplying ` X'*X` and ` X'*y`. These operations are already
multi-threaded in the BLAS version I have installed. It is possible
first calling ` using LinearAlgebra; BLAS.set_num_threads(1)` would
improve the performance of the multi-threaded bootstrap.

In [17]:
println("Single thread, fastest version")
@btime  (b,t) = gridbootstrap(estimator(), a->a, αgrid, nboot);

println("$(nthreads()) threads, fastest version")
estimator_threaded(y0=y0,e=est.e)=function(foo)
  (a, rng) = foo
  out=simulate_estimate_arp(y0,a,e,Val(1),()->rand(rng,1:length(e)))
  (out.θ[3], out.se[3])
end
rng = rngarray(nthreads())
@btime begin
  (bs, ts) = gridbootstrap_threaded(estimator_threaded(),
                                    (a,rng)->(a,rng), αgrid,
                                    nboot, rng=rng)  
end;

Single thread, fastest version
  26.709 ms (111651 allocations: 4.80 MiB)
30 threads, fastest version
  4.685 ms (21196 allocations: 1.16 MiB)


Notice how the speedup from using multiple threads is far less than
number of cores. On my computer, the threaded version of the code is
about 4 times faster, even though my computer has 40 "cores" (or 20
physical cores. My computer has 2 processors with 10 cores each, and each
core is hyperthreaded into 2. The OS sees 40 processors, but half of
them are sharing substantial resources). A speedup far less than the
number of cores is typical. Creating and managing multiple threads
creates some overhead. Moreover, cores must share various resources;
most notably RAM and some cache. 

# GPU

Compared to CPUs, GPUs have a huge number of cores operating at a
slower clockrate. GPUs also have their own separate memory, which they
can access faster than CPUs access RAM.  These characteristics make
GPUs well-suited to large parallel computations. Unfortunately, fully
utilizing GPUs can require substantial changes to your code.

See ["The Different Flavors of
Parallelism"](https://mitmath.github.io/18337/lecture6/styles_of_parallelism)
Rackauckas (2019)[@rackauckas2019c] for more information comparing
GPUs to various forms of parallelism on CPUs.

## Array interface

The easiest way to use a GPU in Julia is through a high level array
interface. ` ArrayFire.jl`, ` CLArrays.jl`, and ` CuArrays.jl` each offer such
interfaces. We will focus on ` CuArrays.jl` in these
notes. ` CuArrays.jl` relies on Nvidia's CUDA platform, so it only
works with Nvidia GPUs. Nvidia tends to dominate GPGPU, and the GPUs
available on cedar.computecanada.ca and in my desktop are Nvidia. 

Using CuArrays is simple, but has some limitations. You create arrays
on the GPU using ` CuArray`. Any array level operation on these will
then be performed efficiently on the GPU. This includes broadcast
functions with ` .` and matrix multiplies.

In [18]:
using CuArrays, Random, BenchmarkTools


N = 1000
M = 1000

function cuarraydemo(N,M)
  # wrapped in a  function so that the CuArrays are freed
  # otherwise we will run out GPU memory later
  A = randn(N,M);
  b = randn(M,2);
  println("Time on CPU")
  function foo(A,b)
    (A.^2)*b
  end
  @time c=foo(A,b);
  @time c=foo(A,b);
  A_gpu = CuArray(A); # copy of A in GPU memory
  b_gpu = CuArray(b);
  println("Computations on the GPU are fast")
  # @btime does not work inside a function
  @time CuArrays.@sync c_gpu=foo(A_gpu,b_gpu);
  @time CuArrays.@sync c_gpu=foo(A_gpu,b_gpu);
  println("But copying to and from GPU memory is not")
  bar(A,b) =Array(foo(CuArray(A), CuArray(b))) 
  @time c2=bar(A,b);
  @time c2=bar(A,b);  
end

cuarraydemo (generic function with 1 method)

In [19]:
cuarraydemo(N,M);

Time on CPU
  0.004194 seconds (3 allocations: 7.645 MiB)
  0.003710 seconds (3 allocations: 7.645 MiB)
Computations on the GPU are fast


┌ Info: Building the CUDAnative run-time library for your sm_61 device, this might take a while...
└ @ CUDAnative /home/paul/.julia/packages/CUDAnative/Lr0yj/src/compiler/rtlib.jl:173


  7.388847 seconds (11.16 M allocations: 578.044 MiB, 4.29% gc time)
  0.000518 seconds (76 allocations: 2.844 KiB)
But copying to and from GPU memory is not
  0.008507 seconds (98 allocations: 15.308 MiB)
  0.008805 seconds (101 allocations: 15.310 MiB)


` CuArrays` also allow indexing, so you could use loops and other
constructs. However, this will not be fast. ` CuArrays` by itself will be
a good method to utilize GPUs when the code is dominated by operations
on large arrays. 

Unfortunately, the fastest version of our grid bootstrap code does not
fit that description. A loop seems needed to generate $y$ due to the
recursiveness of the AR(1) model. The fastest version of the code
above involves many operations on small 3x3 arrays.

EXERCISE: modify ` b_est_original` or ` b_est_mldivide` to utilize
` CuArrays`. The approach taken in those functions involves some
moderate sized matrices, so it may benefit from ` CuArrays`.


## CUDAnative

To parallelize the code above on a GPU, we will have to use a lower
level interface to the GPU. We will the ` CUDAnative.jl` package. To
explain how it works, we will begin with a simple example that just
squares all the elements of an array. 

Disclaimer: my understanding of CUDA and the inner workings of GPUs is
far from complete. Some of the details in this section might be
inaccurate. 

A typical workflow with CUDAnative consists of 

1. Allocate GPU memory and copying arrays into it with ` CuArray`.
2. Decide how many threads and what configuration of threads to
   launch.
3. Each thread does some computation by running a "kernel" function.
4. Copy result from GPU memory to CPU memory.

In the code below, 1 happens in `cuarray_cudanative_compare`, 2 happens in the
` square!` function, ` square_kernel!` is the kernel in 3, and 4 is just
not done. 

### Threads and blocks

CUDA organizes GPU threads into blocks. I believe that the threads in
a block all execute concurrently. Threads in the same block share some
memory and registers. All current Nvidia GPUs have a maximum number of
threads per block of 1024. Note that threads in the same block share
registers[^reg], and different kernel functions will use different
numbers of registers at once, so depending on the kernel function, you
might be limited to fewer than 1024 threads per block. The number of registers
available per block depends on your GPU. You can check your GPU
characteristics by compiling and running the C++ program in 
`$CUDA_PATH/samples/1_Utilities/deviceQuery/`. 
Alternatively, you can access this information within Julia using
` CUDAdrv.jl`.

In [20]:
using CUDAdrv
println("Maximum threads per block $(attribute(device(), CUDAdrv.MAX_THREADS_PER_BLOCK))")
println("Maximum x blocks $(attribute(device(), CUDAdrv.MAX_GRID_DIM_X))")
println("Maximum registers per block $(attribute(device(), CUDAdrv.MAX_REGISTERS_PER_BLOCK))")

Maximum threads per block 1024
Maximum x blocks 2147483647
Maximum registers per block 65536


As far as I can tell, there is no simple way to figure out how many
registers a kernel function uses. It will depend both on the code you
write and how the compiler optimizes the code. If you encounter
cryptic error messages about CUDA resources unavailable, then try
reducing the number of threads per block.

You can execute more than 1024 threads by specifying a number of
blocks. There is also a limit to the number of blocks, but it is
rather large. In the code below, we set the number of blocks, so that
` nblocks*nthreads >= length(A)`. Each thread then operates on a single
element of ` A`. When the code is executed, each thread has a unique
` threadIdx` and ` blockIdx` combination, and these are used to assign
threads to elements of ` A`. The indices go from 1 to number of threads
(or blocks). For convenience you can request threads and blocks to
have up 3 dimensions, and there are ` threadIdx().y` and
` threadIdx().z` for the additional dimensions.

[^reg]: Processor registers are the fastest bits of memory on the
    processor, and registers are where the actual addition,
    multiplication, and other instructions are carried out.

In [21]:
using CUDAnative

function square!(A::CuArray)
  n = length(A)
  maxthreads = 1024
  nthreads = min(maxthreads, n)
  nblocks  = Int(ceil(n/nthreads))

  @cuda threads=nthreads blocks=nblocks square_kernel!(A)
  
  return A
end

function square_kernel!(A)
  i = threadIdx().x + (blockIdx().x-1)*blockDim().x
  if (i<=length(A))
    @inbounds A[i] *= A[i]
  end
  return nothing # CUDA kernels must return nothing
end

function cuarray_cudanative_compare(A)
  A_gpu = CuArray(A);
  println("CUDAnative square!")
  @time CuArrays.@sync square!(A_gpu);
  @time CuArrays.@sync square!(A_gpu);

  println("CuArray A*=A")
  A_gpu = CuArray(A);
  @time CuArrays.@sync A_gpu .*= A_gpu;
  @time CuArrays.@sync A_gpu .*= A_gpu;
  return nothing
end

cuarray_cudanative_compare (generic function with 1 method)

In [22]:
cuarray_cudanative_compare(randn(N,M))

CUDAnative square!
  0.144017 seconds (100.71 k allocations: 6.322 MiB)
  0.000426 seconds (21 allocations: 592 bytes)
CuArray A*=A
  0.226973 seconds (214.06 k allocations: 10.758 MiB)
  0.000303 seconds (53 allocations: 2.594 KiB)


### Kernel Limitations

CUDA kernel functions execute on the GPU and in GPU memory. Since GPU
memory is allocated and managed differently than RAM, many Julia
functions will not work in CUDA kernels. Most importantly, Julia
functions that allocate dynamically sized arrays will not work. This
means that even matrix multiplication like ` θ = ixx*xy` will fail (if
`ixx` or `xy` are dynamically allocated) since it allocates an array
for ` θ`. You can, however, have local scalars, tuples, and `
StaticArrays` within a kernel function. The key difference is that the
sizes of these types are known at compile time. If `ixx` and `xy` are
`StaticArrays`, then you can do something like `θ = ixx*xy`. Since the
compiler knows the size of `ixx` and `xy`, the compiler also know the
size of `θ`. However, even with ` StaticArrays` you must be careful
with operations that that create new StaticArrays (like matrix
multiplies). These will cause problems if called repeatedly within a
loop.[^loops]

[^loops]: If you create StaticArrays inside a loop, they get allocated
    to the GPU's "dynamic shared memory." I believe a new allocation
    happens each loop iteration. This will be slow, and there is a
    fairly small amount of dynamic shared memory, of which you will
    soon run out.

It is possible to dynamicaaly allocate GPU memory within a kernel
function, but it requires using the low-level interface to CUDA in 
`CUDAnative.jl` and/or ` CUDAdrv.jl`. Moreoever, it is generally not a
good idea to be dynamically allocating and freeing memory in each of
the thousands of threads you execute.[^caveat]


[^caveat]: There are situations where allocating shared memory is a
    needed and a good idea, but these require some advanced techniques
    that we will not cover.

## GPU grid bootstrap

In [23]:
println.(functiontext("argridbootstrap_gpu",joinpath(dirname(Base.pathof(ARGridBootstrap)), "gridbootstrap.jl")));

"""
    argridbootstrap_gpu(e; αgrid = 0.84:(0.22/20):1.06,
                          nboot=199, RealType = Float32)

Computes grid bootstrap estimates for an AR(1) model. 

For each α ∈ grid, repeatedly simulate data with parameter α and then compute an estimate. 
 
# Arguments
- `e` vector error terms that will be resampled with replacement
        to generate bootstrap sample  
- `grid` grid of parameter values. For each value, `nboot`
        datasets will be simulated and estimates computed.  
- `nboot` 
- `RealType` type of numbers for GPU computation. On many GPUs,
        Float32 will have better performance than Float64.

# Returns
- `ba` hatα - α for each grid value and simulated dataset
- `t` t-stat  for each grid value and simulated dataset   
"""
function argridbootstrap_gpu(e, y0;
                             grid = 0.84:(0.22/20):1.06,
                             nboot=199, RealType = Float32)
  g = length(grid)

  P = 3
  # Allocate GPU memory
  bootq = CuArray(zeros(R

In [24]:
println.(functiontext("argridkernel!",joinpath(dirname(Base.pathof(ARGridBootstrap)), "gridbootstrap.jl")));

"""
    argridkernel!(ba,bootq, bootse, ar::Val{P}, e, ei, αgrid) 

GPU kernel for simulation and estimation of AR(P) model. 

# Arguments (modified on return)
- `ba`: `nboot × ngrid` array.  Will be filled with bootstrap estimates of α
   grid values of true α 
- `bootq`: `nboot × ngrid` array.  Will be filled with bootstrap
   estimates of α
- `bootse`: `nboot × ngrid` array.  Will be filled with standard
   errors of α for each bootstrap repetition      


# Arguments (not modified)
- `ar::Val{P}` : autoregressive order for estimation. Simulated
   model will always be AR(1) with 0 intercept and time trend, but
   estimation will use an AR(P) model with intercept and time
   trend. Only the AR(1) parameter estimate is included in `ba`,
   `bootq`, and `bootse`.
- `e` : error terms to draw with replacement
- `ei` : `nboot × ngrid × length(e)` array of indices of `e` to
         use to generate bootstrap sample1
- `αgrid` : length `ngrid` values of AR(1) parameter to perform
   bootst

In [25]:
@btime begin
  grid = argridbootstrap_gpu(est.e, y0, grid=αgrid, nboot=nboot, RealType=Float64);
end;

  1.071 ms (289 allocations: 892.50 KiB)


Compared to the fastest CPU code above, the GPU version takes about
1/20th the time of the single-threaded CPU code, and about 1/5th the
time of the 30-threaded CPU code. Considering that the two CPUs in my
workstation together cost about 6 times more than the single GPU, the
performance of the GPU code is quite good. Also, we carefully profiled
and tuned the CPU code, but not the GPU code (although the GPU code
does use all algorithmic improvements of the fastest CPU code). Profiling GPU kernel
code requires using Nvidia's profiler, see
[CUDAnative
documentation](https://juliagpu.github.io/CUDAnative.jl/stable/man/performance.html)
for information.